# Clasificación de metafora por medio de CNN utilizando el modelo Roberta-large:

A partir de este modelo preentrenado se generan los embeddings correspondientes a cada oración del corpus Trofi, una vez generados los vectores se ajustan a una dimensión de (1, 1024)  y se propone una red convolucional (1 dimensión) de una capa conectada a una capa densamente conectada como clasificador.

Las etiquetas de cada oración se presentan en formato one hot:

0 --> [1, 0] --> Texto literal

1 --> [0, 1] --> Texto metaforico

Es importante aclarar que el funcionamiento de la red se probó con hasta 6 capas convolucionales pero su desempeño es mejor con una sola capa convolucional.


*Proyecto de tésis 2022, para el grado de Maestría en Ciencias en Ingenieria del Computo.
CIC-IPN*

*Alumna: Ericka Deyanira Ovando B.*

In [ ]:
from google.colab import drive #para importar y exportar archivos desde drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf #se importan las librerias a utilizar
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
#%matplotlib inline

In [ ]:
import pickle
data = pickle.load(open('/content/drive/MyDrive/SEROBERTABASE',  'rb'))
x_train_total = data[0]
y_train_onehot_total= data[1]
sentence_train_total= data[2]
x_test_total= data[3]
y_test_onehot_total= data[4]
sentence_test_total= data[5]
#se importan los datos almacenados en pickle correspondientes a los conjuntos de
#k- fold- cross- validation de senteces embeddings generados a partir de Roberta-large del dataset Trofi

In [ ]:
x_train= x_train_total[0]
y_train_onehot=y_train_onehot_total[0]
sentence_train=sentence_train_total[0]
x_test= x_test_total[0]
y_test_onehot= y_test_onehot_total[0]
sentence_test= sentence_test_total[0]

In [ ]:
x_train_total[0][1].shape #forma de x

TensorShape([1, 768])

In [ ]:
y_test_onehot_total[0][0] #forma de las etiquetas

array([1., 0.])

# Para utilizar la GPU disponible en colaboratory:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 24 07:41:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |    438MiB / 15109MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Se diseña la red convolucional utilizando TF:

In [ ]:
#Cargamos la versión de TF a usar

import tensorflow as tf  #Se revisa si está disponible GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#Declaramos la arquitectura de la red...
class DNN_model(object):
  def __init__(self,
               n_classes=2):
    #Declaramos los pesos y bías de cada capa
    #[128, 128, 3]-> [64,64, 64]->[32,32,128 ]->[16,16,128]
    self.h1LW = tf.Variable(np.random.rand(5,1,32),name="hl1weigths",dtype="float32") #capa conv 1
    self.h1LB = tf.Variable(np.random.rand(32),name="hl1bias",dtype="float32")

    self.h2LW = tf.Variable(np.random.rand(5,32,64),name="hl2weigths",dtype="float32")  #capa coonv 2
    self.h2LB = tf.Variable(np.random.rand(64),name="hl2bias",dtype="float32")

    self.h3LW = tf.Variable(np.random.rand(5,64,128),name="hl2weigths_O",dtype="float32") #capa conv 3
    self.h3LB = tf.Variable(np.random.rand(128),name="hl2bias_O",dtype="float32")

    self.h4LW = tf.Variable(np.random.rand(5,128,256),name="hl2weigths_O",dtype="float32") #capa conv 3
    self.h4LB = tf.Variable(np.random.rand(256),name="hl2bias_O",dtype="float32")

    self.h5LW = tf.Variable(np.random.rand(5,256,512),name="hl2weigths_O",dtype="float32") #capa conv 3
    self.h5LB = tf.Variable(np.random.rand(512),name="hl2bias_O",dtype="float32")

    self.h6LW = tf.Variable(np.random.rand(5,512,512),name="hl2weigths_O",dtype="float32") #capa conv 3
    self.h6LB = tf.Variable(np.random.rand(512),name="hl2bias_O",dtype="float32")

    self.h7LW = tf.Variable(np.random.rand(1*96*128,1024),name="hl3weigths",dtype="float32") #capa aplanar vector 4
    self.h7LB = tf.Variable(np.random.rand(1024),name="hl3bias",dtype="float32")

    self.outW = tf.Variable(np.random.rand(1024, n_classes),name="outweigths",dtype="float32") #capa conectada 5

    self.outB = tf.Variable(np.random.rand(n_classes),name="outbias",dtype="float32")
    self.trainable_variables =[self.h1LW,self.h1LB,
                               self.h2LW,self.h2LB,
                               self.h3LW,self.h3LB,
                               self.h4LW,self.h4LB,
                               self.h5LW,self.h5LB,
                               self.h6LW,self.h6LB,
                               self.h7LW,self.h7LB,
                               self.outW,self.outB]

  def __call__(self,x, rate):

      x  = tf.cast(x, tf.float32)
      print('x', x.shape)
      #El formato es N -> datos en el batch
      # H -> height
      # W -> width
      # C -> channels
      # [N, H, W, C]

      img = tf.reshape(x, shape=[-1, 1, 768, 1])
      print('img:', img.shape)

      #primera capa conv ---> [1, 1024]
      l1 = tf.add(img, self.h1LB)
      l1= tf.nn.conv1d(l1,self.h1LW, stride= 1, padding='SAME')
      mean_1,std_1 = tf.nn.moments(l1, axes=[0,1,2])
      l1= tf.nn.batch_normalization(l1, mean_1, std_1, None, None, 1e-12)
      l1 = tf.nn.relu(l1)

      l1 = tf.nn.max_pool(l1, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')
      print('l1', l1.shape)

      #segunda capa conv ---> [1, 512]
      l2= tf.nn.conv1d(l1,self.h2LW, stride= 1, padding='SAME')
      l2 = tf.add(l2, self.h2LB)
      mean_2,std_2 = tf.nn.moments(l2, axes=[0,1,2])
      l2=tf.nn.batch_normalization(l2, mean_2, std_2, None, None, 1e-12)
      l2 = tf.nn.relu(l2)
      l2 = tf.nn.max_pool(l2, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')
      print('l2', l2.shape)

      #tercera capa conv 3 ---> [1, 256]
      l3= tf.nn.conv1d(l2,self.h3LW, stride= 1, padding='SAME')
      l3 = tf.add(l3, self.h3LB)
      mean_3,std_3 = tf.nn.moments(l3, axes=[0,1,2])
      l3 =tf.nn.batch_normalization(l3, mean_3, std_3, None, None, 1e-12)
      l3 = tf.nn.relu(l3)
      l3 = tf.nn.max_pool(l3, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')
      print('l3', l3.shape)

      #cuarta capa conv 4 ---> [1, 128]
      l4= tf.nn.conv1d(l3,self.h4LW, stride= 1 , padding='SAME')
      l4 = tf.add(l4, self.h4LB)
      mean_4,std_4 = tf.nn.moments(l4, axes=[0,1,2])
      l4 =tf.nn.batch_normalization(l4, mean_4, std_4, None, None, 1e-12)
      l4 = tf.nn.relu(l4)
      l4 = tf.nn.max_pool(l4, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')
      print(l4.shape, 'l4')

      #quinta capa conv 4 ---> [1, 64]
      l5= tf.nn.conv1d(l4,self.h5LW, stride= 1, padding='SAME')
      l5 = tf.add(l5, self.h5LB)
      mean_5,std_5 = tf.nn.moments(l5, axes=[0,1,2])
      l5 =tf.nn.batch_normalization(l5, mean_5, std_5, None, None, 1e-12)
      l5 = tf.nn.relu(l5)
      l5 = tf.nn.max_pool(l5, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')
      print(l5.shape, 'l5')

      #sexta capa conv 4 ---> [1, 32]
      l6= tf.nn.conv1d(l5,self.h6LW, stride= 1, padding='SAME')
      l6 = tf.add(l6, self.h6LB)
      mean_6,std_6 = tf.nn.moments(l6, axes=[0,1,2])
      l6 =tf.nn.batch_normalization(l6, mean_6, std_6, None, None, 1e-12)
      l6 = tf.nn.relu(l6)
      l6 = tf.nn.max_pool(l6, ksize=[1,1,2,1], strides=[1,1,2,1], padding='SAME')
      print(l6.shape, 'l6')
      print( 'input shape:', l6.shape)

      # sexta capa para aplanar vector  -> 1* 64* 128
      l7=  tf.reshape(l2,[-1, 1*96*128])
      l7=  tf.matmul(l7,self.h7LW)
      l7= tf.add(l7, self.h7LB)
      print(l7.shape)

      l7 = tf.nn.relu(l7)
      l7=tf.nn.dropout(l7, rate)

      output = tf.matmul(l7,self.outW) + self.outB
      print('output', len(output))
      print(output.shape)
      return output


In [ ]:
DNN = DNN_model()
DNN(x_train[1600: 1610], 0.6)

x (10, 1, 768)
img: (10, 1, 768, 1)
l1 (10, 1, 384, 32)
l2 (10, 1, 192, 64)
l3 (10, 1, 96, 128)
(10, 1, 48, 256) l4
(10, 1, 24, 512) l5
(10, 1, 12, 512) l6
input shape: (10, 1, 12, 512)
(10, 1024)
output 10
(10, 2)


<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[489258.25, 498035.4 ],
       [467475.3 , 448858.62],
       [552937.8 , 552084.56],
       [580223.7 , 548106.6 ],
       [512560.56, 487258.66],
       [474876.6 , 474440.34],
       [606512.4 , 560655.94],
       [532882.3 , 501608.12],
       [504580.28, 484304.6 ],
       [649120.5 , 633171.06]], dtype=float32)>

In [ ]:
from os import listdir # se cargan las librerias necesarias
from os.path import isfile, join
import numpy as np
import cv2
import matplotlib.pyplot as plt
import datetime, os

In [ ]:
#Cargamos un oprtimizador, como siempre, usaremos el de cajón con un learning rate comun
optimizador = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)
#optimizador= tf. compat. v1.train.AdagradOptimizer(learning_rate=0.001)
#optimizador= tf.compat.v1.train.AdadeltaOptimizer(learning_rate=.001)

In [ ]:
#Declaramos las metricas, para la perdida y el accuracy...
#Para training
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
#Para testing
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')






In [ ]:
#Hacemos nuestra fnción del paso de entrenamiento...
@tf.function
def train_step(model,tdata, labels, rate):
  with tf.GradientTape() as tape:
    #Se propagan los datos y se obtiene una salida...
    predictions = model(tdata, rate)

    #calculo de una funcion de error
    #cce = tf.keras.losses.CategoricalCrossentropy()
    #loss= tf.reduce_mean(cce(labels, predictions))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, predictions))

  #Calculamos  los gradientes
  gradients = tape.gradient(loss, model.trainable_variables)
  #ordenamos los parametros con su gradiente...
  capped_grads_and_vars = [(grad,model.trainable_variables[index]) for index, grad in enumerate(gradients)]
  #Aplicamos nuestro optimizador seleccionado
  optimizador.apply_gradients(capped_grads_and_vars)
  #perdida
  train_loss(loss)
  #accuracy...
  train_accuracy(labels, predictions)


In [ ]:
#Nuestro paso de prueba...
#Nuestro paso de prueba...
@tf.function
def test_step(model,tdata, labels):
  #Hacemos la propagacion de los datos y obtenemos la salida
  predictions = model(tdata, 0) #para probar el rate=0
  #Calulamos el error
  #t_cce = tf.keras.losses.CategoricalCrossentropy()
  #t_loss= tf.reduce_mean(t_cce(labels, predictions))
  t_loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, predictions))

  # error
  test_loss(t_loss)
  # accuracy
  test_accuracy(labels, predictions)








In [ ]:
test_step(DNN,x_test[0:10],y_test_onehot[0:10])

x (10, 1, 768)
img: (10, 1, 768, 1)
l1 (10, 1, 384, 32)
l2 (10, 1, 192, 64)
l3 (10, 1, 96, 128)
(10, 1, 48, 256) l4
(10, 1, 24, 512) l5
(10, 1, 12, 512) l6
input shape: (10, 1, 12, 512)
(10, 1024)
output 10
(10, 2)


In [ ]:
#Aqui agregamos el registro de todo lo que queremos almacenar
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #Un marcador del tiempo actual

train_log_dir = '/content/drive/MyDrive/logs/gradient_tape/' + current_time + '/train' #registro de entrenamiento
test_log_dir = '/content/drive/MyDrive/logs/gradient_tape/' + current_time + '/test' #Hregistro de pruebas
train_summary_writer = tf.summary.create_file_writer(train_log_dir) #escriba de entrenamiento
test_summary_writer = tf.summary.create_file_writer(test_log_dir) # escriba de pruebas


In [ ]:
#Declaramos la funcion de entrenamiento
def fitting(model,train_x,train_y,test_x,test_y,EPOCHS,N_batch,batch_size, rate):
  """
  Esta función lleva a cabo el entrenamiento de una red neuronal
  Recibe:
  * model.- La red neuronal a entrenar. Debe ser un objeto de la clase "DNN_model"
  * train_x.- El conjunto de datos de entrenamiento.
  * train_y.- El conjutno de etiquetas de entrenamiento.
  * test_x.- El conjunto de datos de prueba.
  * test_y.- El conjunto de etiquetas de prueba.
  * EPOCHS.- El número de épocas de entrenamiento.
  * N_batch.- El número de lotes (batches) a procesar
  * bacth_size.- El tamaño de cada lote (batch)

  No entrega nada como salida pero muestra en pantalla el proceso del entrenamiento
  """
  #El ciclo de numero de epocas
  img = np.reshape(train_x[0], (-1, 1, 768, 1))
  with train_summary_writer.as_default():

    tf.summary.image("Training data", img, step=0)

  for epoch in range(EPOCHS):
    i=0
    #El ciclo por los batches...
    while i+batch_size < len(train_x): #or i+batch_size<batch_size*N_batch:
      #Calculamos el inicio y fin del lote...
      start = i
      end = i+batch_size
      #Tomamos la rebanada en train_x y train_y a propagar por la red...
      batch_x = train_x[start:end]
      batch_y = train_y[start:end]
      #Hacemos el paso de entrenamiento, enviamos la red, los datos y sus etiquetas...
      train_step(model,batch_x,batch_y, rate)
      #Hacemos el incremento en el contador del lote...
      i+=batch_size
    #Hacemos el paso de prueba...
    with train_summary_writer.as_default():
      #tf.summary.scalar('loss', train_loss.result(), step=epoch)
      tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)



    tf.summary.trace_on(graph=True, profiler=True)
    test_step(model,test_x,test_y)





    with train_summary_writer.as_default():
      tf.summary.trace_export(
        name="function_trace", step=0, profiler_outdir= train_log_dir)

    with test_summary_writer.as_default():
      tf.summary.scalar('loss', test_loss.result(), step=epoch)
      tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
      #tf.summary.scalar('fp', test_fp.result(), step=epoch)
      #tf.summary.scalar('fn', test_fn.result(), step=epoch)
      #tf.summary.scalar('tp', test_tp.result(), step=epoch)
      #tf.summary.scalar('tn', test_tn.result(), step=epoch)
      #tf.summary.scalar('precisión', test_precision.result(), step=epoch)
      #tf.summary.scalar('recall', test_recall.result(), step=epoch)


    #Desplegamos en pantalla la epoca, la pérdida y el error de entrenamient y de prueba...
    template = 'Epoch {}, Perdida: {}, Exactitud: {}, Perdida de prueba: {}, Exactitud de prueba: {}' #,Fp {}, Fn {}, Tp {}, Tn {}, presición {}, recall {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result(),
                        test_loss.result(),
                        test_accuracy.result(),
                        #test_fp.result(),
                        #test_fn.result(),
                        #test_tp.result(),
                        #test_tn.result(),
                        #test_precision.result(),
                        #test_recall.result(),
                        ))

    #Reseteamos las metricas para que no se acumule la información de todas las épocas...
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    #test_fp.reset_states()
    #test_fn.reset_states()
    #test_tp.reset_states()
    #test_tn.reset_states()
    #test_precision.reset_states()
    #test_recall.reset_states()



# Se entrena la red y se prueba con los diferentes conjuntos k- fold validation para evaluar su desempeño.

In [ ]:
#Declaramos la funcion de entrenamiento
def fitting2(model,train_x,train_y,test_x,test_y,EPOCHS,N_batch,batch_size, rate):
  """
  Esta función lleva a cabo el entrenamiento de una red neuronal
  Recibe:
  * model.- La red neuronal a entrenar. Debe ser un objeto de la clase "DNN_model"
  * train_x.- El conjunto de datos de entrenamiento.
  * train_y.- El conjutno de etiquetas de entrenamiento.
  * test_x.- El conjunto de datos de prueba.
  * test_y.- El conjunto de etiquetas de prueba.
  * EPOCHS.- El número de épocas de entrenamiento.
  * N_batch.- El número de lotes (batches) a procesar
  * bacth_size.- El tamaño de cada lote (batch)

  No entrega nada como salida pero muestra en pantalla el proceso del entrenamiento
  """
  #El ciclo de numero de epocas
  img = np.reshape(train_x[0], (-1, 1, 512, 1))
  with train_summary_writer.as_default():

    tf.summary.image("Training data", img, step=0)

  for epoch in range(EPOCHS):
    i=0
    #El ciclo por los batches...
    while i+batch_size < len(train_x): #or i+batch_size<batch_size*N_batch:
      #Calculamos el inicio y fin del lote...
      start = i
      end = i+batch_size
      #Tomamos la rebanada en train_x y train_y a propagar por la red...
      batch_x = train_x[start:end]
      batch_y = train_y[start:end]
      #Hacemos el paso de entrenamiento, enviamos la red, los datos y sus etiquetas...
      train_step(model,batch_x,batch_y, rate)
      #Hacemos el incremento en el contador del lote...
      i+=batch_size
    #Hacemos el paso de prueba...
    with train_summary_writer.as_default():
      tf.summary.scalar('loss', train_loss.result(), step=epoch)
      tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)



    tf.summary.trace_on(graph=True, profiler=True)
    test_step(model,test_x,test_y)
    print('x', test_x[1], 'y', test_y[1])


    with train_summary_writer.as_default():
      tf.summary.trace_export(
        name="function_trace", step=0, profiler_outdir= train_log_dir)

    with test_summary_writer.as_default():
      tf.summary.scalar('loss', test_loss.result(), step=epoch)
      tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    #Desplegamos en pantalla la epoca, la pérdida y el error de entrenamient y de prueba...
    template = '{}'
    print(template.format(
                        test_accuracy.result()))

    #Reseteamos las metricas para que no se acumule la información de todas las épocas...
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    test_fn.reset_state()


In [ ]:
fitting(DNN,x_train,y_train_onehot,x_test,y_test_onehot,400,100,200, .7)

x (200, 1, 768)
img: (200, 1, 768, 1)
l1 (200, 1, 384, 32)
l2 (200, 1, 192, 64)
l3 (200, 1, 96, 128)
(200, 1, 48, 256) l4
(200, 1, 24, 512) l5
(200, 1, 12, 512) l6
input shape: (200, 1, 12, 512)
(200, 1024)
output 200
(200, 2)
x (200, 1, 768)
img: (200, 1, 768, 1)
l1 (200, 1, 384, 32)
l2 (200, 1, 192, 64)
l3 (200, 1, 96, 128)
(200, 1, 48, 256) l4
(200, 1, 24, 512) l5
(200, 1, 12, 512) l6
input shape: (200, 1, 12, 512)
(200, 1024)
output 200
(200, 2)


Instructions for updating:
use `tf.profiler.experimental.start` instead.


x (373, 1, 768)
img: (373, 1, 768, 1)
l1 (373, 1, 384, 32)
l2 (373, 1, 192, 64)
l3 (373, 1, 96, 128)
(373, 1, 48, 256) l4
(373, 1, 24, 512) l5
(373, 1, 12, 512) l6
input shape: (373, 1, 12, 512)
(373, 1024)
output 373
(373, 2)


Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


Epoch 1, Perdida: 30438.265625, Exactitud: 0.5803124904632568, Perdida de prueba: 13944.4853515625, Exactitud de prueba: 0.582245409488678
Epoch 2, Perdida: 35195.05859375, Exactitud: 0.3634375035762787, Perdida de prueba: 594.177978515625, Exactitud de prueba: 0.5764074921607971
Epoch 3, Perdida: 10145.0244140625, Exactitud: 0.47593748569488525, Perdida de prueba: 13520.5419921875, Exactitud de prueba: 0.5710455775260925
Epoch 4, Perdida: 19305.224609375, Exactitud: 0.21250000596046448, Perdida de prueba: 3932.623291015625, Exactitud de prueba: 0.5710455775260925
Epoch 5, Perdida: 10499.7216796875, Exactitud: 0.32624998688697815, Perdida de prueba: 6847.58154296875, Exactitud de prueba: 0.5710455775260925
Epoch 6, Perdida: 11880.712890625, Exactitud: 0.2809374928474426, Perdida de prueba: 3857.843505859375, Exactitud de prueba: 0.5764074921607971
Epoch 7, Perdida: 9222.736328125, Exactitud: 0.32249999046325684, Perdida de prueba: 4251.5009765625, Exactitud de prueba: 0.576407492160797

In [ ]:
#Declaramos la funcion de entrenamiento
def fitting2(model,train_x,train_y,test_x,test_y,EPOCHS,N_batch,batch_size, rate):
  """
  Esta función lleva a cabo el entrenamiento de una red neuronal
  Recibe:
  * model.- La red neuronal a entrenar. Debe ser un objeto de la clase "DNN_model"
  * train_x.- El conjunto de datos de entrenamiento.
  * train_y.- El conjutno de etiquetas de entrenamiento.
  * test_x.- El conjunto de datos de prueba.
  * test_y.- El conjunto de etiquetas de prueba.
  * EPOCHS.- El número de épocas de entrenamiento.
  * N_batch.- El número de lotes (batches) a procesar
  * bacth_size.- El tamaño de cada lote (batch)

  No entrega nada como salida pero muestra en pantalla el proceso del entrenamiento
  """
  #El ciclo de numero de epocas
  img = np.reshape(train_x[0], (-1, 1, 512, 1))
  with train_summary_writer.as_default():

    tf.summary.image("Training data", img, step=0)

  for epoch in range(EPOCHS):
    i=0
    #El ciclo por los batches...
    while i+batch_size < len(train_x): #or i+batch_size<batch_size*N_batch:
      #Calculamos el inicio y fin del lote...
      start = i
      end = i+batch_size
      #Tomamos la rebanada en train_x y train_y a propagar por la red...
      batch_x = train_x[start:end]
      batch_y = train_y[start:end]
      #Hacemos el paso de entrenamiento, enviamos la red, los datos y sus etiquetas...
      train_step(model,batch_x,batch_y, rate)
      #Hacemos el incremento en el contador del lote...
      i+=batch_size
    #Hacemos el paso de prueba...
    with train_summary_writer.as_default():
      tf.summary.scalar('loss', train_loss.result(), step=epoch)
      tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)



    tf.summary.trace_on(graph=True, profiler=True)
    test_step(model,test_x,test_y)
    #print('x', test_x[1], 'y', test_y[1])


    with train_summary_writer.as_default():
      tf.summary.trace_export(
        name="function_trace", step=0, profiler_outdir= train_log_dir)

    with test_summary_writer.as_default():
      tf.summary.scalar('loss', test_loss.result(), step=epoch)
      tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    #Desplegamos en pantalla la epoca, la pérdida y el error de entrenamient y de prueba...
    template = '{}'
    print(template.format(
                        test_accuracy.result()))

    #Reseteamos las metricas para que no se acumule la información de todas las épocas...
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    test_fn.reset_state()


In [ ]:
#K=9
#x_train= x_train_total[K]
#y_train_onehot=y_train_onehot_total[K]
#sentence_train=sentence_train_total[K]
#x_test= x_test_total[K]
#y_test_onehot= y_test_onehot_total[K]
#sentence_test= sentence_test_total[K]

for i in range (10):
  K=i
  x_train= x_train_total[K]
  y_train_onehot=y_train_onehot_total[K]
  #sentence_train=sentence_train_total[K]
  x_test= x_test_total[K]
  y_test_onehot= y_test_onehot_total[K]
  #sentence_test= sentence_test_total[K]

  fitting2(DNN,x_train,y_train_onehot,x_test,y_test_onehot,1,100,200, .7)

ValueError: ignored

In [ ]:
def metricas(x, y, rate, k):

  ##se declaran las métricas
  test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
  test_fn= tf.keras.metrics.FalseNegatives(name= 'test_fn')
  test_fp=tf.keras.metrics.FalsePositives(name= 'test_fp')
  test_tn= tf.keras.metrics.TrueNegatives(name= 'test_tn')
  test_tp=tf.keras.metrics.TruePositives(name='test_tp')
  test_precision =tf.keras.metrics.Precision(name='test_precision')
  test_recall=tf.keras.metrics.Recall(name='test_recall')


  pred= DNN(x, rate)
  new=[]
  lit=0
  met=0

  ##0 es literal y 1 es metafora
  for i in range(len(pred)):
    if pred[i][0]>=pred[i][1]:
      new.append(0)
      lit= lit+1
    if pred[i][0]<pred[i][1]:
      new.append(1)
      met= met+1

  labels= []
  for j in range (len(y)):
    if y_test_onehot[j][0]==1.:
      labels.append(0)
      met= met+1
    else:
      labels.append(1)
      lit= lit+1

  print(labels[0:10])
  print(new[0:10])

  print(len(x_test), met, lit)


  test_accuracy(y_test_onehot, pred)
  test_fn.update_state(labels, new)
  test_fp.update_state(labels, new)
  test_tp.update_state(labels, new)
  test_tn.update_state(labels, new)
  test_precision.update_state(labels, new)
  test_recall.update_state(labels, new)

  print('Para k:', k)
  print('Acuracy', test_accuracy.result())
  print('False negative', test_fn.result())
  print('False positive', test_fp.result())
  print('True Negative', test_tn.result())
  print('True positive', test_tp.result())
  print('Presición', test_precision.result())
  print('recall', test_recall.result())

  #print(labels[1], new[1])


  test_accuracy.reset_states()
  test_fp.reset_states()
  test_fn.reset_states()
  test_tp.reset_states()
  test_tn.reset_states()
  test_precision.reset_states()
  test_recall.reset_states()

In [ ]:
for i in range (10):
  K=i
  x_train= x_train_total[K]
  y_train_onehot=y_train_onehot_total[K]
  #sentence_train=sentence_train_total[K]
  x_test= x_test_total[K]
  y_test_onehot= y_test_onehot_total[K]
  #sentence_test= sentence_test_total[K]

  metricas(x_test,y_test_onehot, .7, K)

x (373, 1, 768)
img: (373, 1, 768, 1)
l1 (373, 1, 384, 32)
l2 (373, 1, 192, 64)
l3 (373, 1, 96, 128)
(373, 1, 48, 256) l4
(373, 1, 24, 512) l5
(373, 1, 12, 512) l6
input shape: (373, 1, 12, 512)
(373, 1024)
output 373
(373, 2)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
373 407 339
Para k: 0
Acuracy tf.Tensor(0.7211796, shape=(), dtype=float32)
False negative tf.Tensor(35.0, shape=(), dtype=float32)
False positive tf.Tensor(69.0, shape=(), dtype=float32)
True Negative tf.Tensor(91.0, shape=(), dtype=float32)
True positive tf.Tensor(178.0, shape=(), dtype=float32)
Presición tf.Tensor(0.72064775, shape=(), dtype=float32)
recall tf.Tensor(0.8356807, shape=(), dtype=float32)
x (373, 1, 768)
img: (373, 1, 768, 1)
l1 (373, 1, 384, 32)
l2 (373, 1, 192, 64)
l3 (373, 1, 96, 128)
(373, 1, 48, 256) l4
(373, 1, 24, 512) l5
(373, 1, 12, 512) l6
input shape: (373, 1, 12, 512)
(373, 1024)
output 373
(373, 2)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 1, 0, 0, 0, 1]
373 424 322
P